In [2]:
import numpy as np
import h5py
import os
import requests
import tempfile
import time

import scann

2022-10-19 20:19:10.809392: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-19 20:19:11.130638: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/mesa-diverted/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu/mesa:/usr/lib/x86_64-linux-gnu/dri:/usr/lib/x86_64-linux-gnu/gallium-pipe
2022-10-19 20:19:11.130698: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-19 20:19:11.179574: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS facto

In [3]:
with tempfile.TemporaryDirectory() as tmp:
    response = requests.get("http://ann-benchmarks.com/glove-100-angular.hdf5")
    loc = os.path.join(tmp, "glove.hdf5")
    with open(loc, 'wb') as f:
        f.write(response.content)
    
    glove_h5py = h5py.File(loc, "r")

In [4]:
list(glove_h5py.keys())

['distances', 'neighbors', 'test', 'train']

In [5]:
dataset = glove_h5py['train']
queries = glove_h5py['test']
print(dataset.shape)
print(queries.shape)

(1183514, 100)
(10000, 100)


In [14]:
normalized_dataset = dataset / np.linalg.norm(dataset, axis=1)[:, np.newaxis]
# configure ScaNN as a tree - asymmetric hash hybrid with reordering
# anisotropic quantization as described in the paper; see README

# use scann.scann_ops.build() to instead create a TensorFlow-compatible searcher
searcher = scann.scann_ops_pybind.builder(normalized_dataset, 1, "dot_product").tree(
    num_leaves=2000, num_leaves_to_search=100, training_sample_size=250000).score_ah(
    2, anisotropic_quantization_threshold=0.2).reorder(100).build()

2022-10-19 20:28:16.387135: I scann/partitioning/partitioner_factory_base.cc:59] Size of sampled dataset for training partition: 249797
2022-10-19 20:28:25.078566: I ./scann/partitioning/kmeans_tree_partitioner_utils.h:88] PartitionerFactory ran in 8.691306582s.


In [15]:
neighbors, distances = searcher.search_batched(queries)


In [8]:
print(list(glove_h5py))

['distances', 'neighbors', 'test', 'train']


In [16]:
print(neighbors[100])

[718095]


In [17]:
print(glove_h5py['train'][1])

[-0.99544   -2.3651    -0.24332   -1.0321     0.42052   -1.1817
 -0.16451   -1.683      0.49673   -0.27258   -0.025397   0.34188
  1.5523     1.3532     0.33297   -0.0056677 -0.76525    0.49587
  1.2211     0.83394   -0.20031   -0.59657    0.38485   -0.23487
 -1.0725     0.95856    0.16161   -1.2496     1.6751     0.73899
  0.051347  -0.42702    0.16257   -0.16772    0.40146    0.29837
  0.96204   -0.36232   -0.47848    0.78278    0.14834    1.3407
  0.47834   -0.39083   -1.037     -0.24643   -0.75841    0.7669
 -0.37363    0.52741    0.018563  -0.51301    0.97674    0.55232
  1.1584     0.73715    1.3055    -0.44743   -0.15961    0.85006
 -0.34092   -0.67667    0.2317     1.5582     1.2308    -0.62213
 -0.032801   0.1206    -0.25899   -0.02756   -0.52814   -0.93523
  0.58434   -0.24799    0.37692    0.86527    0.069626   1.3096
  0.29975   -1.3651    -0.32048   -0.13741    0.33329   -1.9113
 -0.60222   -0.23921    0.12664   -0.47961   -0.89531    0.62054
  0.40869   -0.08503    0.6413